In [1]:
import pandas as pd
import numpy as np
from time import time
import json
from sqlalchemy import create_engine
import re
import datetime
import lifetimes
#import pymc3 as pm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from statsmodels.tsa.arima_process import ArmaProcess
from causalimpact import CausalImpact

In [2]:
# Set up the SQL connection
dbname = 'redshift'
url = 'panamera-ods.cnsuxis6zqxr.us-west-2.redshift.amazonaws.com:5439/panameraods'
engine = create_engine("postgresql://" + 'lucasdresl' + ":" + 'cQsY801a76x82x62aWDT' + "@" + url)

In [ ]:
query = """
;drop table if exists treatment_B;
select
  session_long,
  date_event_nk,
  time_event_utc
  into temp table treatment_B
    from ods.panameraolx_asia_hydra_ninja_android_last_month
  where
    trackevent = 'messages_scheduler_ended'
group by 1,2,3

;drop table if exists users_step_1;
select
  date_event_nk,
  count(distinct session_long) as starter
  into temp table users_step_1
    from treatment_B
 group by 1

;drop table if exists push_show;
select
  a.session_long,
  a.date_event_nk,
  a.time_event_utc as time_scheduler,
  c.time_event_utc as time_push_arrive,
  datediff(minute , c.time_event_utc , a.time_event_utc)
  into temp table push_show
    from treatment_B a
      join (select session_long, date_event_nk, time_event_utc from ods.panameraolx_asia_hydra_ninja_android_last_month where trackevent = 'push_show' and date_event_nk > '2019-07-10' group by 1,2,3) c on a.session_long=c.session_long and a.date_event_nk=c.date_event_nk
  where
    datediff(minute , c.time_event_utc , a.time_event_utc) <= 10 and datediff(minute , c.time_event_utc , a.time_event_utc) >= 0


;drop table if exists users_step_2;
select
    date_event_nk,
  count(distinct session_long) as push_show
  into temp table users_step_2
    from push_show
 group by 1

;drop table if exists convirtio_en_dau;
select
  a.session_long,
  a.date_event_nk
   into temp table convirtio_en_dau
  from push_show a
    join (select session_long , date_nk from ods.fact_user_hydra_browsing_activity where country_sk = 'olx|asia|in' and is_panamera = TRUE and date_nk > '2019-07-10' group by 1,2) b on a.session_long=b.session_long and a.date_event_nk=b.date_nk
group by 1,2

;drop table if exists users_step_3;
select
    date_event_nk,
  count(distinct session_long) as dau
  into temp table users_step_3
    from convirtio_en_dau
 group by 1;

select
  date_event_nk,
  a.starter,
  b.push_show,
  c.dau
       from users_step_1 a
left join users_step_2 b on a.date_event_nk=b.date_event_nk
left join users_step_3 c on a.date_event_nk=c.date_event_nk


"""

In [ ]:
df = pd.read_sql_query(query.replace('%', '%%'), engine)

In [ ]:
df.head(20)

In [ ]:
## tato approac

In [3]:
query = """
drop table if exists tabla1;
      select  session_long,
              test_name,
              test_variation as variation,
              min (date_event_nk) as first_exposition
       into temp table tabla1
      from ods.panameraolx_asia_hydra_ninja_android_last_month
      where country_sk='olx|asia|in'
      and trackevent='test_impression'
      and test_name like 'pan-31472'
      and test_variation is not null
      and date_event_nk >= '2019-07-11'
      group by 1,2,3


;drop table if exists tabla2;
select  a.session_long,
       a.test_name,
       a.variation,
       a.first_exposition ,
       b.date_event_nk,
       b.test_variation,
       count(distinct case when b.test_variation is not null then b.session_long
               else null end) as count_dups
    into temp tabla2
 from tabla1 as a
left join
       (select session_long,date_event_nk,test_variation
       from ods.panameraolx_asia_hydra_ninja_android_last_month
       where country_sk='olx|asia|in'
      and trackevent='test_impression'
      and test_name like 'pan-31472'
      and test_variation is not null
      and date_event_nk >= '2019-07-11') as b
on a.session_long=b.session_long and b.date_event_nk>=first_exposition and a.variation != b.test_variation
group by 1,2,3,4,5,6


;drop table if exists session_long_table;
select
  a.session_long,
  a.date_event_nk,
  b.first_exposition,
  b.variation,
  case when a.date_event_nk>b.first_exposition then 1 else 0 end as user_exposure
  into temp table session_long_table
   from ods.panameraolx_asia_hydra_ninja_android_last_month a
     join (select session_long,first_exposition,variation from tabla2 where count_dups=0 group by 1,2,3) b on a.session_long=b.session_long
 where
      date_event_nk > '2019-07-11'
  group by 1,2,3,4,5


;drop table if exists tabla3;
select
  session_long,
  date_event_nk as date_scheduler_seen
   into temp table tabla3
   from ods.panameraolx_asia_hydra_ninja_android_last_month
    where trackevent='messages_scheduler_ended'
 group by 1,2


;drop table if exists push_show;
select
  session_long,
  date_event_nk as date_push_receiver,
  time_event_utc
   into temp table push_show
   from ods.panameraolx_asia_hydra_ninja_android_last_month
    where trackevent='push_show'
 group by 1,2,3

;drop table if exists tabla4;
select
  a.*,
  case when b.date_scheduler_seen is not null then 1 else 0 end as treatment,
  case when c.date_push_receiver is not null then 1 else 0 end as recibio_push,
  c.time_event_utc as time_receiving_push
  into temp tabla4
    from session_long_table a
      left join tabla3 b on a.session_long=b.session_long and a.date_event_nk=b.date_scheduler_seen
      left join push_show c on a.session_long=c.session_long and a.date_event_nk=c.date_push_receiver;

    select
      a.*,
      b.time_event_utc,
      datediff(minute , a.time_receiving_push , b.time_event_utc) as datediff,
      case when (datediff(minute , a.time_receiving_push , b.time_event_utc) <= 1440 and datediff(minute , a.time_receiving_push , b.time_event_utc) >= 0) or b.session_long is not null then 1 else 0 end as convirtio_en_dau
       into temp table last_step_funnel
       from tabla4 a
         left join (select c.session_long , date_event_nk , time_event_utc
                     from ods.panameraolx_asia_hydra_ninja_android_last_month c
                       join (select session_long from tabla4 group by 1) t on t.session_long=c.session_long
                         where trackevent not in ('app_open', 'on_create', 'apps', 'push_dis','pushDis', 'push_rcv', 'pushRcv', 'UAReg', 'ua_reg', 'uareg','push_dismissed', 'push_show','fetching_matrix', 'not_found','permissions_impression','userBadgeCount','test_assignment','test_impression','test_discovered','FCMReg','APNSReg','appOpn','appOpen','google_play_services','fetchTestDefinitionsError','messages_scheduler_ended','messages_scheduler_started')
                     and date_event_nk>'2019-07-10' group by 1,2,3) b on a.session_long=b.session_long


"""

In [4]:
df = pd.read_sql_query(query.replace('%', '%%'), engine)

OperationalError: (psycopg2.errors.QueryCanceled) Query (37330153) canceled on user's request and ran out of wlm queues for restart.

[SQL: 
drop table if exists tabla1;
      select  session_long,
              test_name,
              test_variation as variation,
              min (date_event_nk) as first_exposition
       into temp table tabla1
      from ods.panameraolx_asia_hydra_ninja_android_last_month
      where country_sk='olx|asia|in'
      and trackevent='test_impression'
      and test_name like 'pan-31472'
      and test_variation is not null
      and date_event_nk >= '2019-07-11'
      group by 1,2,3


;drop table if exists tabla2;
select  a.session_long,
       a.test_name,
       a.variation,
       a.first_exposition ,
       b.date_event_nk,
       b.test_variation,
       count(distinct case when b.test_variation is not null then b.session_long
               else null end) as count_dups
    into temp tabla2
 from tabla1 as a
left join
       (select session_long,d